# Conda Envrionment install instructions

1. Open anaconda prompt -> cd C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo
2. Create a folder where you want the env files to be --> mkdir venv
3. conda create --prefix "C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv" python=3.11.6
4. conda activate "C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv"
5. conda install -p "C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv" ipykernel --update-deps --force-reinstall
6. cd Scene-Recognition
7. pip install -r Requirements.txt
8. pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118 

# Data Loader Check

In [ ]:
from dataloader import get_data_loader
import torchvision

train_split = 0.75
full_dataset = torchvision.datasets.ImageFolder(root="Data/")
train_size = int(train_split * len(full_dataset))
test_size = int((len(full_dataset) - train_size)/2)
val_size = len(full_dataset) - train_size - test_size

train_size, test_size, val_size

In [ ]:
train_dataloader, test_dataloader, val_dataloader = get_data_loader(batch_size=64,data_dir="Data/", shuffle=True)
for a in train_dataloader:
    print(a[0].shape, a[1].shape)

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [ ]:
import torch

torch.cuda.is_available()

# Model Training

In [ ]:
# # pip install split-folders

# import splitfolders
# splitfolders.ratio('Data/', output="output", seed=1337, ratio=(.7, 0.15,0.15)) 

In [1]:
# import packages 
import os 
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

import torch 
import torch.nn as nn

#import your model here
from log import create_logger
from dataloader import get_data_loader, get_data_loader_split
from models.resnet import resnet18
from models.efficientnet import effnet_s
from models.VGG import VGG
from datetime import datetime

# Add your models here
models = {'resnet18': resnet18,
         'enet_s':effnet_s,
         'vgg':VGG
         }

# RUN DETAILS
# run_name = "jly_0206_enets_lr1e-6_bs=64"

now = datetime.now()
timestamp = now.strftime("%Y-%m-%d_%H%M%S")
run_name = f"run_{timestamp}"

print(run_name)  # Output: run_2024-02-08_143154


model_base = 'enet_s'
num_epochs = 250
bs = 64
lr = 1e-6
random_seed = 42
save_chks = range(num_epochs) # iterable of epochs for which to save the model
use_split = True
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
if device == 'mps':
    torch.mps.empty_cache()

# set up run dir 
run_dir = os.path.join('saved_models', run_name)
os.makedirs(run_dir, exist_ok = True)
log, logclose = create_logger(log_filename=os.path.join(run_dir, 'train.log'), display = False)
log(f'using device: {device}')
log(f'saving models to: {run_dir}')
log(f'using base model: {model_base}')
log(f'using batch size: {bs}')
log(f'learning rate: {lr}')
log(f'random seed: {random_seed}')

# seed randoms and make deterministic
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
# random.seed(random_seed)
torch.backends.cudnn.enabled=False
torch.backends.cudnn.deterministic=True

# dataloader
if use_split==True:
    train_dataloader, test_dataloader, val_dataloader = get_data_loader_split(data_dir="output/",  batch_size=bs, shuffle=True)
else:
    train_dataloader, test_dataloader, val_dataloader = get_data_loader(data_dir="Data/",  batch_size=bs, shuffle=True)

# define model 
model = models[model_base]()
model.to(device)

# define optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# training loop
train_loss = []
val_loss = []
test_loss = []
train_metrics = []
val_metrics = []
for epoch in range(num_epochs):
    print(f"epoch: {epoch}")
    log(f'epoch {epoch}')
    #training
    model.train()
    batch_loss = []
    batch_metric = []
    total_imgs = 0
    for i, (_data, _target) in tqdm(enumerate(train_dataloader)): 
        data = _data.to(device)
        target = _target.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
        batch_metric.append(sum(torch.argmax(pred, dim=1)==target).item())
        total_imgs += len(target)
    train_loss.append(sum(np.array(batch_loss)/len(train_dataloader)))
    log(f'\ttrain loss: {train_loss[-1]}')
    train_metrics.append(sum(batch_metric)/total_imgs) #TODO: add metrics
    print(f"Training Metric --- Train Accuracy: {sum(batch_metric)/total_imgs} ---- Train Loss: {sum(np.array(batch_loss)/len(train_dataloader))}")
    del data 
    del target
    del pred
    del loss

    # validation
    total_imgs = 0
    batch_metric = []
    with torch.no_grad():
        model.eval()
        batch_loss = []
        for i, (_data, _target) in tqdm(enumerate(val_dataloader)): 
            data = _data.to(device)
            target = _target.to(device)
            pred = model(data)
            loss = criterion(pred, target)
            batch_loss.append(loss.item())
            batch_metric.append(sum(torch.argmax(pred, dim=1)==target).item())
            total_imgs += len(target)
        val_loss.append(sum(np.array(batch_loss)/len(val_dataloader)))
        log(f'\tval loss: {val_loss[-1]}')
        val_metrics.append(sum(batch_metric)/total_imgs) #TODO: add metrics
        print(f"Validation Metric --- Val Accuracy: {sum(batch_metric)/total_imgs} ---- Val Loss: {sum(np.array(batch_loss)/len(val_dataloader))}")


    if epoch in save_chks: 
        torch.save(model.state_dict(), os.path.join(run_dir, f'{epoch}.chkpt'))

    plt.plot(train_loss, label='train')
    plt.plot(val_loss, label='val')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend()
    plt.savefig(os.path.join(run_dir, 'loss'))
    plt.close()
    plt.plot(train_metrics, label='train accuracy')
    plt.plot(val_metrics, label='val accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend()
    plt.savefig(os.path.join(run_dir, 'accu'))
    plt.close()
    del data 
    del target
    del pred
    del loss

    if device == 'mps':
        torch.mps.empty_cache()


# testing
with torch.no_grad():
    model.eval()

run_2024-02-08_161108
epoch: 0


64it [00:28,  2.27it/s]


Training Metric --- Train Accuracy: 0.1564039408866995 ---- Train Loss: 1.6164458077400923


14it [00:05,  2.63it/s]


Validation Metric --- Val Accuracy: 0.14959723820483314 ---- Val Loss: 1.6137045281273978
epoch: 1


...

Training Metric --- Train Accuracy: 0.6509852216748768 ---- Train Loss: 1.4744126237928867


14it [00:04,  2.82it/s]


Validation Metric --- Val Accuracy: 0.6962025316455697 ---- Val Loss: 1.4755907910210746
epoch: 151


22it [00:08,  2.63it/s]


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    model.eval()
    batch_loss = []
    for i, (_data, _target) in tqdm(enumerate(test_dataloader)): 
        data = _data.to(device)
        target = _target.to(device)
        pred = model(data)
        loss = criterion(pred, target)
        batch_loss.append(loss.item())
        batch_metric.append(sum(torch.argmax(pred, dim=1)==target).item()/len(target))
    test_loss.append(sum(np.array(batch_loss)/len(test_dataloader)))
    log(f'\tval loss: {val_loss[-1]}')
    val_metrics.append(np.mean(batch_metric)) #TODO: add metrics

# Predict from images

In [11]:
# import torch
import PIL as pil





# #load image
# #resize to 224x224
# org_image = pil.Image.open('/Users/rishabhshah/Desktop/AIPI 590/Scene-Recognition/output/test/cathedral/gsun_1a8814d41a9e6565b1b947d8b79c4c39.jpg', mode='r')


# model.predict('/Users/rishabhshah/Desktop/AIPI 590/Scene-Recognition/saved_models/run_2024-02-10_150948/resnet18_full_model_0.pt')


import torchvision.transforms as transforms
import torch



# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize(224),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485,  0.456,  0.406], std=[0.229,  0.224,  0.225]),
])

# Load and preprocess the image
org_image = pil.Image.open('/Users/rishabhshah/Desktop/AIPI 590/Scene-Recognition/output/test/cathedral/gsun_1a8814d41a9e6565b1b947d8b79c4c39.jpg')
input_tensor = preprocess(org_image)
input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

# Perform inference
with torch.no_grad():
    output = model(input_batch.to('mps'))

# Get the predicted class
_, predicted = torch.max(output,  1)
classes = ['Campsite','Candy Store','Canyon','Castle','Cathedral']

print(f'Predicted class: {classes[predicted.item()]}')




Predicted class: Castle


# Loading model

In [1]:
import torch